morphology.py is a pipeline that calls the statmorph package.  This notebook shows how to use morphology.py in a notebook or interactive setting.

The morphology pipeline requires only one argument at the command line.  The argument is the filename of the stamp you want to analyze.  You will also need a segmentation stamp, i.e., a cutout from the segmentation map that corresponds to the cutout to be analyzed.  The segmentation stamp must use 0 to indicate the absence of source flux in a pixel.  (PyBDSM uses -1 to indicate background in its `island images'.)

Let's use stamp.py to create the stamps and segmentation stamps:

In [9]:
import subprocess
subprocess.call(['../stamps/stamp.py', '--radio_img',
                 '../radio_imgs/img_MACSJ0416_S-BAND_data-MFS-image.fits',
                 '--cat', '../ian_cats/VLA-HFF_0416_compact_optical_rasort.txt', '--imgs',
                 '../radio_imgs/pyrank/img_MACSJ0416_S-BAND_data-MFS-image.pybdsm_pyrank.fits'])

0

The results of the call are stamps from the radio data (\*img0.fits) and stamps from the segmentation image (\*img1.fits).  The pipeline, run from the command line, requires that the segmentation stamp share the root name of the data stamp and end in .seg.fits.  For this example the segmentation stamps must be \*img0.seg.fits.  So let's rename the files:

In [10]:
subprocess.call(['for i in output/*img1.fits; do mv "$i" "${i%%img1.fits}img0.seg.fits"; done'],
                shell = True)

0

Now, import morphology and read in 1 pair of stamps for an example:

In [11]:
from astropy.io import fits
import morphology
img = fits.open('output/113img0.fits')
segm = fits.open('output/113img0.seg.fits')
# pyrank images indicate background with -1; we must indicate background with 0
segm[0].data += 1

morphology.py has 5 main functions: return_bkg, measure_morphology, mod_segmap_1pix, write_gini_mask, and build_table.

The first step is to get the median and RMS of the background in the stamp.  statmorph requires that the stamp has a background of 0.  The segmentation stamp defines the masked region.  The median and RMS are calculated with sigma_clipped_stats.

The inputs to return_bkg are the stamp and the segmentation stamp as a boolean mask.  The outputs are the median of the masked and sigma-clipped region and a 2D array of the same size as the stamp filled with the RMS value.  (We assume the RMS is constant across the stamp.)

In [12]:
bkg_median, bkg_rms_arr = morphology.return_bkg(img[0].data, segm[0].data.astype(bool))

In [13]:
print(bkg_median, bkg_rms_arr)

(1.4006300830260443e-07, array([[  9.84536169e-07,   9.84536169e-07,   9.84536169e-07, ...,
          9.84536169e-07,   9.84536169e-07,   9.84536169e-07],
       [  9.84536169e-07,   9.84536169e-07,   9.84536169e-07, ...,
          9.84536169e-07,   9.84536169e-07,   9.84536169e-07],
       [  9.84536169e-07,   9.84536169e-07,   9.84536169e-07, ...,
          9.84536169e-07,   9.84536169e-07,   9.84536169e-07],
       ..., 
       [  9.84536169e-07,   9.84536169e-07,   9.84536169e-07, ...,
          9.84536169e-07,   9.84536169e-07,   9.84536169e-07],
       [  9.84536169e-07,   9.84536169e-07,   9.84536169e-07, ...,
          9.84536169e-07,   9.84536169e-07,   9.84536169e-07],
       [  9.84536169e-07,   9.84536169e-07,   9.84536169e-07, ...,
          9.84536169e-07,   9.84536169e-07,   9.84536169e-07]]))


The second step is to pass to measure_morphology the stamp, segmentation stamp, median background, background RMS array, and a "cutout extent" parameter, which is passed to statmorph.  You can fiddle with the extent parameter but I've used 2.

The outputs of measure_morphology are the morphological parameters from statmorph for the object labeled in the central pixel of the segmentation stamp.

In [14]:
obj_morph = morphology.measure_morphology(img[0].data, segm[0].data, bkg_median, bkg_rms_arr, 2)

Finished processing source 550.



In [16]:
print(obj_morph[0])

If obj_morph exists, the pipeline will call statmorph an additional 2 times: once with a segmentation stamp whose boundary has been expanded by 1 pixel; and once with a segmentation stamp whose boundary has been shrunk by 1 pixel.  The idea is to estimate the morphological parameters 3 times, under segmentation uncertainty, and then to calculate a standard deviation.

mod_segmap_1pix modifies the segmentation stamp.

In [17]:
big_segm, small_segm = morphology.mod_segmap_1pix(segm[0].data)